#Redes Neuronales 2021

Integrantes de grupo:

Müller, Malena

Scala, Tobías 
# TP2: Sesgos en el dataset de SNLI

El trabajo practico consiste en predecir las ventas diarias para los proximos 6 meses de Rossmann, que opera en 3000 locales en 7 paises europeos. Las ventas son influenciadas por varios factores; incluyendo promociones, competencia, el periodo escolar, fechas festivas, temporada y ubicacion. 


## Importacion de datos

En esta primera etapa, levantamos los datos provenientes de los archivos de train, test y store, que corresponden al periodo de tiempo a los años entre el 2013 y el 2015. Luego chequeamos si faltan datos en ciertos campos de cada linea que fue levantada de los archivos.

In [85]:
import pandas as pd
import numpy as np
#import datetime
#from isoweek import Week

trainFile = pd.read_csv('train.csv')
#print(trainFile.columns)
testFile = pd.read_csv('test.csv')
#print(trainFile.columns)
storeFile = pd.read_csv('store.csv')
#print(trainFile.columns)

#Me fijo si faltan datos en TRAIN.
print('train[Customers]: NA = ', sum(trainFile['Customers'].isna()))
print('train[Open]: NA = ', sum(trainFile['Open'].isna()))
print('train[Promo]: NA = ', sum(trainFile['Promo'].isna()))
print('train[StateHoliday]: NA = ', sum(trainFile['StateHoliday'].isna()))
print('train[SchoolHoliday]: NA = ', sum(trainFile['SchoolHoliday'].isna()))

#Me fijo si faltan datos en TEST.
# print('test[Customers]: NA = ', sum(testFile['Customers'].isna())) #No tiene Customers.
print('test[Open]: NA = ', sum(testFile['Open'].isna())) #11
print('test[Promo]: NA = ', sum(testFile['Promo'].isna()))
print('test[StateHoliday]: NA = ', sum(testFile['StateHoliday'].isna()))
print('test[SchoolHoliday]: NA = ', sum(testFile['SchoolHoliday'].isna()))

#Me fijo si faltan datos en STORE.
print('store[StoreType]: NA = ', sum(storeFile['StoreType'].isna()))
print('store[Assortment]: NA = ', sum(storeFile['Assortment'].isna()))
print('store[CompetitionDistance]: NA = ', sum(storeFile['CompetitionDistance'].isna())) #3
print('store[Promo2]: NA = ', sum(storeFile['Promo2'].isna()))

# print(trainFile['StateHoliday'].value_counts())
# print(sum(trainFile['StateHoliday'] == 0))
# print(sum(trainFile['StateHoliday'] == '0'))

# trainFile['StateHoliday'] = trainFile['StateHoliday'].astype(str)

# print(trainFile['StateHoliday'].value_counts())
# print(sum(trainFile['StateHoliday'] == 0))
# print(sum(trainFile['StateHoliday'] == '0'))

# print(storeFile['CompetitionOpenSinceYear'].value_counts())
# print(len(storeFile['CompetitionOpenSinceYear']))
# print(storeFile['CompetitionDistance'].max())

C:\Users\Tobias Scala\anaconda3\envs\rn\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


train[Customers]: NA =  0
train[Open]: NA =  0
train[Promo]: NA =  0
train[StateHoliday]: NA =  0
train[SchoolHoliday]: NA =  0
test[Open]: NA =  11
test[Promo]: NA =  0
test[StateHoliday]: NA =  0
test[SchoolHoliday]: NA =  0
store[StoreType]: NA =  0
store[Assortment]: NA =  0
store[CompetitionDistance]: NA =  3
store[Promo2]: NA =  0


## Preprocesamiento de datos

En cada data set levantado previamente, completamos con los datos faltantes teniendo en cuenta los siguientes criterios:
- En el campo "Open": Este campo puede tomar los valores de 1 (Abierto) y 0 (Cerrado). En caso de que no se haya indicado nada en este campo, se lo considera como cerrado (0).
- En el campo "Competition Distance": Dado que hay negocios de competencia que no existian previamente a la fecha de inicio de datos, solo hay informacion de distancia a la que se encuentra esa competencia a partir del momento en el que se establecio en su lugar geografico. Como necesitamos que haya informacion correspondiente a la distancia a la que se encuentra la competencia desde el inicio de fecha de los datos del data set, es necesario completar con informacion las fechas que no tienen dato. Si un local de competencia no existe aun, no tiene impacto en las ventas de nuestros locales y lo mismo ocurre con un local de competencia que geograficamente se encuentra muy lejos. Por eso se decidio tomar como criterio que para las fechas en las que la competencia aun no existia, la distancia sea completada con un valor muy grande de distancia (como si estuviera muy lejos). Este valor elegido es el maximo encontrado en el dataset.
- Tanto en el campo "CompetitionOpenSinceYear" como en el campo "CompetitionOpenSinceMonth": En caso de que no haya informacion en estos campos, se asume que la competencia ya existia para la fecha de los primeros datos del dataset, por lo que se le asigna a cada campo el dato minimo presente en el dataset. 



In [86]:
#Completamos los datos en todos los archivos.

#trainFile.loc[trainFile['Open'].isna(), 'Open'] = 0
testFile.loc[testFile['Open'].isna(), 'Open'] = 0

storeFile.loc[storeFile['CompetitionDistance'].isna(), 'CompetitionDistance'] = storeFile['CompetitionDistance'].max()
storeFile.loc[storeFile['CompetitionOpenSinceYear'].isna(), 'CompetitionOpenSinceYear'] = storeFile['CompetitionOpenSinceYear'].min()
storeFile.loc[storeFile['CompetitionOpenSinceMonth'].isna(), 'CompetitionOpenSinceMonth'] = storeFile['CompetitionOpenSinceMonth'].min()

# print(trainFile['Date'][0])


En esta parte pasamos las fechas al formato de fechas necesario para los analisis siguientes, empleando una funcion de Pandas. Pasamos los datos numericos a int32 con la finalidad de poder interpretarlos luego como numeros. Y borramos la columna (o campo) de "Costumers" presente en los datos levantados de train, ya que en test no estaban estos datos y buscamos que haya consistencia entre los campos de datos que tenemos de train y de test.



In [87]:
#Empiezamos a preprocesar los datos.

import datetime

trainFile['Date'] = pd.to_datetime(trainFile.Date)
testFile['Date'] = pd.to_datetime(testFile.Date)
storeFile['CompetitionOpenSinceYear'] = storeFile['CompetitionOpenSinceYear'].astype(np.int32)
storeFile['CompetitionOpenSinceMonth'] = storeFile['CompetitionOpenSinceMonth'].astype(np.int32)
storeFile['CompetitionOpenSince'] = pd.to_datetime(storeFile.apply(lambda x: datetime.datetime(x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 1), axis=1))

trainFile['CompetitionDistance'] = np.zeros(len(trainFile['Date']))
for i in range(len(trainFile['Date'])):
    if(trainFile['Date'][i] > storeFile['CompetitionOpenSince'][trainFile['Store'][i] - 1]):
        trainFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'][trainFile['Store'][i] - 1]
    else:
        trainFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'].max()
#trainFile['StateHoliday'] = ['0' if trainFile['StateHoliday'] == 0 else trainFile['StateHoliday']]
trainFile['StateHoliday'] = trainFile['StateHoliday'].astype(str)
trainFile['SchoolHoliday'] = trainFile.SchoolHoliday != '0' #Convertimos strings a bool (ya que es binario).
#trainFile['SchoolHoliday'] = trainFile['SchoolHoliday'].astype(np.int8)
del trainFile['Customers'] #Borramos la columna costumers ya que el test no lo tiene (?).
#print(trainFile.head())

testFile['CompetitionDistance'] = np.zeros(len(testFile['Date']))
for i in range(len(testFile['Date'])):
    if(testFile['Date'][i] > storeFile['CompetitionOpenSince'][testFile['Store'][i] - 1]):
        testFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'][testFile['Store'][i] - 1]
    else:
        testFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'].max()
#trainFile['StateHoliday'] = ['0' if trainFile['StateHoliday'] == 0 else trainFile['StateHoliday']]
testFile['StateHoliday'] = testFile['StateHoliday'].astype(str)
testFile['SchoolHoliday'] = testFile.SchoolHoliday != '0' #Convertimos strings a bool (ya que es binario).
#testFile['SchoolHoliday'] = testFile['SchoolHoliday'].astype(np.int8)
#print(testFile.head())

<ipython-input-87-7c7ca1c695bc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'][trainFile['Store'][i] - 1]
<ipython-input-87-7c7ca1c695bc>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainFile['CompetitionDistance'][i] = storeFile['CompetitionDistance'].max()
<ipython-input-87-7c7ca1c695bc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testFile['Competit

In [88]:
print(trainFile.head())
print(testFile.head())

   Store  DayOfWeek       Date  Sales  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263     1      1            0   
1      2          5 2015-07-31   6064     1      1            0   
2      3          5 2015-07-31   8314     1      1            0   
3      4          5 2015-07-31  13995     1      1            0   
4      5          5 2015-07-31   4822     1      1            0   

   SchoolHoliday  CompetitionDistance  
0           True               1270.0  
1           True                570.0  
2           True              14130.0  
3           True                620.0  
4           True              29910.0  
   Id  Store  DayOfWeek       Date  Open  Promo StateHoliday  SchoolHoliday  \
0   1      1          4 2015-09-17   1.0      1            0           True   
1   2      3          4 2015-09-17   1.0      1            0           True   
2   3      7          4 2015-09-17   1.0      1            0           True   
3   4      8          4 2015-09-17   1.0 

A continuacion agregamos algunas columnas de datos que se obtuvieron del archivo Store, a los datos de train y test. Los campos agregados son los siguientes:      
- StoreType
- Assortment
- CompetitionDistance
- Promo2


In [89]:
#Juntamos información del storeFile a trainFile y a testFile.

train = trainFile.merge(storeFile, how='left', left_on='Store', right_on='Store', suffixes=("", "_y"))
del train['CompetitionDistance_y']
del train['CompetitionOpenSinceMonth']
del train['CompetitionOpenSinceYear']
#del train['Promo2']
del train['Promo2SinceWeek']
del train['Promo2SinceYear']
del train['PromoInterval']
del train['CompetitionOpenSince']
#print(len(train[train.StoreType.isnull()]))
print(train.head())

test = testFile.merge(storeFile, how='left', left_on='Store', right_on='Store', suffixes=("", "_y"))
del test['Id']
del test['CompetitionDistance_y']
del test['CompetitionOpenSinceMonth']
del test['CompetitionOpenSinceYear']
#del test['Promo2']
del test['Promo2SinceWeek']
del test['Promo2SinceYear']
del test['PromoInterval']
del test['CompetitionOpenSince']
#print(len(test[train.StoreType.isnull()]))
print(test.head())

   Store  DayOfWeek       Date  Sales  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263     1      1            0   
1      2          5 2015-07-31   6064     1      1            0   
2      3          5 2015-07-31   8314     1      1            0   
3      4          5 2015-07-31  13995     1      1            0   
4      5          5 2015-07-31   4822     1      1            0   

   SchoolHoliday  CompetitionDistance StoreType Assortment  Promo2  
0           True               1270.0         c          a       0  
1           True                570.0         a          a       1  
2           True              14130.0         a          a       1  
3           True                620.0         c          c       0  
4           True              29910.0         a          a       0  
   Store  DayOfWeek       Date  Open  Promo StateHoliday  SchoolHoliday  \
0      1          4 2015-09-17   1.0      1            0           True   
1      3          4 2015-09-17   

In [90]:
train.to_csv('trainPreproc.csv', index=False)
test.to_csv('testPreproc.csv', index=False)

In [137]:
import pandas as pd
import numpy as np

train = pd.read_csv('trainPreproc.csv')
#print(train.columns)
test = pd.read_csv('testPreproc.csv')
#print(test.columns)

train['StateHoliday'] = train['StateHoliday'].astype(str)
test['StateHoliday'] = test['StateHoliday'].astype(str)

C:\Users\Tobias Scala\anaconda3\envs\rn\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


A continuacion se normalizan las variables continuas empleando la ecuacion:     
$\frac{x_i-\mu}{\sigma}$
siendo $x_i$ cada uno de los datos, $\mu$ la media y $\sigma$ el desvio estandar.
Tambien se aplica hot encoder a las variables categoricas.


In [138]:
#Normalizamos las variables continuas y hacemos hot encoder a las variables categóricas.

from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import np_utils
from sklearn_pandas import DataFrameMapper

catColumns = ['Store', 'DayOfWeek', 'StateHoliday', 'StoreType', 'Assortment']
contColumns = ['Open', 'CompetitionDistance', 'Promo', 'SchoolHoliday', 'Promo2']

# for i in cont:
#     xmean = train[i].mean()
#     xstd = train[i].std()
#     train[i] = (train[i].values - xmean)/xstd

# enc = LabelEncoder()
# for i in cat:
#     enc.fit(train[i])
#     train_ = enc.transform(train[i])
#     train_ = np_utils.to_categorical(train_)
#     print(train_)

# print(train.head(train))

catEnc = [(i, LabelEncoder()) for i in catColumns]
contScal = [([i], StandardScaler()) for i in contColumns] #Se estandarizan las variables continuas.

catMapp = DataFrameMapper(catEnc) #Mapea la columna que tienen las variables categóricas (string) en un número de índice (números). [a, b, a, d] = [0, 1, 0, 2]
catMapp.fit(train) #Se arma un one hot vector. El fit asigna la cantidad de clases (?)
contMapp = DataFrameMapper(contScal)
contMapp.fit(train) #Calcula la mdeia y el desvío del dataframe. x' = (x-u)/sigma.

train[catColumns] = catMapp.transform(train)
test[catColumns] = catMapp.transform(test)
train[contColumns] = contMapp.transform(train)
test[contColumns] = contMapp.transform(test)
print(train[catColumns])
print(train[contColumns])
print(test[catColumns])
print(test[contColumns])

         Store  DayOfWeek  StateHoliday  StoreType  Assortment
0            0          4             0          2           0
1            1          4             0          0           0
2            2          4             0          0           0
3            3          4             0          2           2
4            4          4             0          0           0
...        ...        ...           ...        ...         ...
1017204   1110          1             1          0           0
1017205   1111          1             1          2           2
1017206   1112          1             1          0           2
1017207   1113          1             1          0           2
1017208   1114          1             1          3           2

[1017209 rows x 5 columns]
             Open  CompetitionDistance     Promo  SchoolHoliday    Promo2
0        0.452399            -0.484440  1.273237            0.0 -1.001128
1        0.452399            -0.517645  1.273237            0.0  0.9

In [139]:
#Embeddings.

# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Embedding, Input, Flatten, Dense, Concatenate

# catDict = {'Store': 50, 'DayOfWeek': 4, 'StateHoliday': 2, 'StoreType': 2, 'Assortment': 3}

# storeIn = Input(shape=(1,), name="storeIn")
# storeEmb = Embedding(1115, catDict['Store'], name="storeEmb")(storeIn)
# storeFlat = Flatten(name="storeFlat")(storeEmb)
# dayofweekIn = Input(shape=(1,), name="dayofweekIn")
# dayofweekEmb = Embedding(7, catDict['DayOfWeek'], name="dayofweekEmb")(dayofweekIn)
# dayofweekFlat = Flatten(name="dayofweekFlat")(dayofweekEmb)
# stateholidayIn = Input(shape=(1,), name="stateholidayIn")
# stateholidayEmb = Embedding(4, catDict['StateHoliday'], name="stateholidayEmb")(stateholidayIn)
# stateholidayFlat = Flatten(name="stateholidayFlat")(stateholidayEmb)
# storetypeIn = Input(shape=(1,), name="storetypeIn")
# storetypeEmb = Embedding(4, catDict['StoreType'], name="storetypeEmb")(storetypeIn)
# storetypeFlat = Flatten(name="storetypeFlat")(storetypeEmb)
# assortmentIn = Input(shape=(1,), name="assortmentIn")
# assortmentEmb = Embedding(4, catDict['Assortment'], name="assortmentEmb")(assortmentIn)
# assortmentFlat = Flatten(name="assortmentFlat")(assortmentEmb)

# openIn = Input(shape=(1,), name="openIn")
# openDen = Dense(1, name="openDen", activation = 'linear')(openIn)
# competitiondistanceIn = Input(shape=(1,), name="competitiondistanceIn")
# competitiondistanceDen = Dense(1, name="competitiondistanceDen", activation = 'linear')(competitiondistanceIn)
# promoIn = Input(shape=(1,), name="promoIn")
# promoDen = Dense(1, name="promoDen", activation = 'linear')(promoIn)
# schoolholidayIn = Input(shape=(1,), name="schoolholidayIn")
# schoolholidayDen = Dense(1, name="schoolholidayDen", activation = 'linear')(schoolholidayIn)
# promo2In = Input(shape=(1,), name="promo2In")
# promo2Den = Dense(1, name="promo2Den", activation = 'linear')(promo2In)

# print(train['Store'].value_counts())
# print(len(train['Store'].value_counts()))

A continuacion, mediante embedding reducimos la cantidad de clases de una variable categorica.

Dado que tenemos 1115 stores como fuente de datos, lo cual es una enorme cantidad, reducimos este numero y nos quedamos con 50. En cambio, para el resto de los campos de datos nos quedamos con la mitad de la cantidad de datos que tenemos originalmente. Este criterio, conocido como "la regla del pulgar", fue obtenida del video del siguiente link:    

https://course18.fast.ai/lessonsml1/lesson12.html




In [140]:
def catEmbeddings(catColumns, catDict):
    catOut = []
    catIn = []
    for i in catColumns:
        catIn_ = Input(shape=(1,))
        catIn.append(catIn_)
        catEmb = Embedding(len(train[i].value_counts()), catDict[i])(catIn_)
        catFlat = Flatten()(catEmb)
        catOut.append(catFlat)
    return catIn, catOut    

def contInput(contColumns):
    contOut = []
    for i in contColumns:
        contIn = Input(shape=(1,))
        contOut.append(contIn)
    return contOut

In [141]:
#Embeddings.

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Dense, Concatenate, Activation, Dropout
from tensorflow.keras.regularizers import l2

catDict = {'Store': 50, 'DayOfWeek': 4, 'StateHoliday': 2, 'StoreType': 2, 'Assortment': 3}

catIn, catOut = catEmbeddings(catColumns, catDict)
contIn = contInput(contColumns)

merged = Concatenate()(catOut + contIn)

## Modelo de procesamiento
Realizamos el procesamiento empleando tanto la regularizacion L2 como Dropout y comparamos los resultados obtenidos a partir de cada uno de los casos.

L2 y Dropout son metodos que se aplican para reducir la complejidad del modelo y disminuir el sobreajuste (overfitting), permitiendo una mayor generalizacion.

- Regularizacion L2 (tambien conocida como la regularizacion de Ridge): Se agrega regularizacion agregandole un termino a la funcion de costo, la cual se busca minimizar. Este termino contiene una sumatoria de los pesos elevados al cuadrado, multiplicada por un hiperparametro. Si los pesos de ese termino que se agrega son grandes (los w), se agranda la funcion de costo y hay que penalizar esos pesos. L2 toma el cuadrado de cada peso y los suma. Este termino tambien tiene un hiperparametro (lambda) que es el mismo para todos los pesos y que determina la importancia que se le da a los pesos. Si lambda es cero, no hay regularizacion. Si es grande, predomina mas ese termino (y tampoco es la idea). Esta regularizacion se aplica a nivel de capa.

- Dropout: Es otra tecnica de regularizacion para reducir el overfitting en redes neuronales. Se desprecian neuronas en forma aleatoria durante el entrenamiento de la red. Para lograr esto, generalmente se multiplica por cero algunas de las activaciones a la salida de una capa.




In [142]:
# def neuralNetwork(merged):
#     lambdaL2 = 1e-3 #Lambda para la regularización L2.
#     model = Dense(1000, kernel_initializer="uniform", kernel_regularizer=l2(lambdaL2))(merged)
#     model = Activation('relu')(model) #'relu' so the cost function has less platous.
#     model = Dense(1, activation='linear')(model) #'linear' so the output is not categorical.
#     model = Model(catIn + contIn, [model])
#     model.compile(optimizer='adam', metrics=['mse'], loss='mse')
#     #model.compile(optimizer=Adam(lr=0.001), metrics=['mse', rmspe], loss='mse')
#     return model

def neuralNetwork(merged):
    lambdaL2 = 1e-3 #Lambda para la regularización L2.
    model = Dense(1000, kernel_initializer="uniform", kernel_regularizer=l2(lambdaL2))(merged)
    model = Activation('relu')(model) #'relu' so the cost function has less platous.
    model = Dropout(0.2)(model)
    model = Dense(1, activation='linear')(model) #'linear' so the output is not categorical.
    model = Model(catIn + contIn, [model])
    model.compile(optimizer='adam', metrics=['mse'], loss='mse')
    #model.compile(optimizer=Adam(lr=0.001), metrics=['mse', rmspe], loss='mse')
    return model

In [143]:
# def neuralNetwork(merged):
#     lambdaL2 = 1e-3 #Lambda para la regularización L2.
#     model = Dense(1000, kernel_initializer="uniform", kernel_regularizer=l2(lambdaL2), activation='relu')(merged) #'relu' so the cost function has less platous.
#     model = Dense(1, activation='linear')(model) #'linear' so the output is not categorical.
#     return Model(catIn + contIn, [model])

## Validacion
Para esta etapa empleamos la metrica RMSPE, para cada uno de los modelos empleados previamente para el procesamiento ya que asi podremos compararlos.

In [144]:
import datetime

train['Date'] = pd.to_datetime(train.Date)
#test['Date'] = pd.to_datetime(test.Date)
valid = train[train.Date >= datetime.datetime(2015, 6, 25)]
train = train[train.Date < datetime.datetime(2015, 6, 25)]
print(len(train)/(len(train)+len(valid)))
print(len(valid)/(len(train)+len(valid)))

0.9594429463364953
0.04055705366350475


In [145]:
def rmspeMetric(valid, predY):
    rmspe = 0
    for i in range(len(predY)):
        if(valid['Sales'][i] != 0):
            rmspe = rmspe + ((valid['Sales'][i] - predY[i])/valid['Sales'][i])**2
    rmspe = np.sqrt(rmspe/len(predY))
    return rmspe
# def rmspeMetric(validY, predY):
#     return np.sqrt((((validY - predY)/validY)**2).mean())

Preparamos los datos de interés para entrenar a modelo. Dichos datos de interés son:
- Store
- DayOfWeek
- StateHoliday
- StoreType
- Assortment
- Open
- CompetitionDistance
- Promo
- SchoolHoliday
- Promo2

In [146]:
columns = catColumns + contColumns
trainX = np.hsplit(train[columns].values, len(columns))
validX = np.hsplit(valid[columns].values, len(columns))
testX = np.hsplit(test[columns].values, len(columns))

print(trainX)
print(validX)
print(testX)

[array([[0.000e+00],
       [1.000e+00],
       [2.000e+00],
       ...,
       [1.112e+03],
       [1.113e+03],
       [1.114e+03]]), array([[2.],
       [2.],
       [2.],
       ...,
       [1.],
       [1.],
       [1.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]]), array([[2.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [3.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [2.],
       [2.],
       [2.]]), array([[ 0.45239852],
       [ 0.45239852],
       [ 0.45239852],
       ...,
       [-2.21044047],
       [-2.21044047],
       [-2.21044047]]), array([[-0.4844404 ],
       [-0.51764533],
       [ 0.12558175],
       ...,
       [-0.10542975],
       [-0.50341465],
       [-0.29090304]]), array([[-0.78539979],
       [-0.78539979],
       [-0.78539979],
       ...,
       [-0.78539979],
       [-0.78539979],
       [-0.78539979]]), array([[0.],
       [0.],
       [0.],
       ...,
       [

Normalizamos los datos correspondientes a la salida de nuestro modelo (Sales) tanto en el datframe del train como de validation.

In [147]:
#print(train.groupby('Store').mean()['Sales'])
trainY = (train['Sales'].values - train.groupby('Store').mean()['Sales'][train['Store']].values)
trainYmax = trainY.max()
trainY = trainY/trainYmax
#trainY = np.hsplit(trainY, trainY.shape[1])
print(trainY)
validY = (valid['Sales'].values - valid.groupby('Store').mean()['Sales'][valid['Store']].values)
validYmax = validY.max()
validY = validY/validYmax
#validY = np.hsplit(validY, validY.shape[1])
print(validY)

[-0.01835703  0.0210813  -0.00853301 ... -0.16756264 -0.52005182
 -0.15797597]
[ 0.05341483  0.06451466  0.08626561 ...  0.00542481 -0.05094384
 -0.04227365]


## Entrenamiento
Se entrena el modelo propuesto con los métodos de regularización mencionados.

In [148]:
model = neuralNetwork(merged)
#model.fit(trainX, trainY, validation_data=(validX, validY),  epochs=20, batch_size=256, verbose=2)
model.fit(trainX, trainY, epochs=20, batch_size=256, verbose=2)

Epoch 1/20
3813/3813 - 26s - loss: 0.0022 - mse: 0.0016
Epoch 2/20
3813/3813 - 27s - loss: 0.0016 - mse: 0.0014
Epoch 3/20
3813/3813 - 27s - loss: 0.0016 - mse: 0.0014
Epoch 4/20
3813/3813 - 28s - loss: 0.0016 - mse: 0.0014
Epoch 5/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 6/20
3813/3813 - 29s - loss: 0.0015 - mse: 0.0014
Epoch 7/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 8/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 9/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 10/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 11/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 12/20
3813/3813 - 26s - loss: 0.0015 - mse: 0.0014
Epoch 13/20
3813/3813 - 26s - loss: 0.0015 - mse: 0.0014
Epoch 14/20
3813/3813 - 27s - loss: 0.0015 - mse: 0.0014
Epoch 15/20
3813/3813 - 24s - loss: 0.0015 - mse: 0.0014
Epoch 16/20
3813/3813 - 23s - loss: 0.0015 - mse: 0.0014
Epoch 17/20
3813/3813 - 24s - loss: 0.0015 - mse: 0.0014
Epoch 18/20
3813/3813 - 32s - loss: 0.00

Evaluamos nuestro modelo con validación ya que se conocen los valores reales de salida (Sales). Este paso es importante ya que, dependiendo que tan bien funciona nuestro modelo, se proseguirá con la predicción de la salida (Sales) con el data frame de test.

In [149]:
loss, acc = model.evaluate(trainX, trainY)
print(acc)

loss, acc = model.evaluate(validX, validY)
print(acc)

30499/30499 [==============================] - 68s 2ms/step - loss: 0.0014 - mse: 0.0013
0.0013035677839070559
1290/1290 [==============================] - 4s 3ms/step - loss: 0.0022 - mse: 0.0021
0.002109960885718465


Se predice la salida (Sales) del data frame de test y luego se lo guarda en un archivo csv para hacer submit en Kaggle.

In [150]:
predY = model.predict(validX, verbose = 1)[:,0]*trainYmax
rmspe = rmspeMetric(valid, predY)
print(rmspe)

predY = model.predict(testX, verbose = 1)[:,0]*trainYmax
# rmspe = rmspeMetric(test, predY)
# print(rmspe)

1290/1290 [==============================] - 2s 2ms/step
0.8174407271455243
1284/1284 [==============================] - 6s 5ms/step


In [151]:
submission = pd.read_csv('sample_submission.csv')
submission['Sales'] = predY
submission.to_csv('submission.csv', index=False)